In [1]:
# Mount drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Define base path

BASE_PATH = '/content/drive/MyDrive/AI/tcfase3'

In [3]:
# Necessary installations

!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.9 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.


In [4]:
# Nacessary Imports

import datasets
from datasets import load_dataset
import re
from bs4 import BeautifulSoup
import math

In [5]:
# Import the Json file to a dataset

dataset = load_dataset('json', data_files=BASE_PATH+'/trn.json')
print(dataset)

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['uid', 'title', 'content', 'target_ind', 'target_rel'],
        num_rows: 2248619
    })
})


In [6]:
# Remove from the dataset variable the columns uid, target_ind and target_rel

dataset = dataset.remove_columns(['uid', 'target_ind', 'target_rel'])
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['title', 'content'],
        num_rows: 2248619
    })
})


In [7]:
# remove lines from the dataset variable that either the title column or content column are empty

def filter_empty_rows(example):
  return example['title'] and example['content']

dataset = dataset.filter(filter_empty_rows)
print(dataset)

Filter:   0%|          | 0/2248619 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['title', 'content'],
        num_rows: 1390403
    })
})


In [8]:
# print the first 20 rows from the dataset variable

for i in range(20):
  print(dataset['train'][i])

{'title': 'Girls Ballet Tutu Neon Pink', 'content': 'High quality 3 layer ballet tutu. 12 inches in length'}
{'title': "Mog's Kittens", 'content': 'Judith Kerr&#8217;s best&#8211;selling adventures of that endearing (and exasperating) cat Mog have entertained children for more than 30 years. Now, even infants and toddlers can enjoy meeting this loveable feline. These sturdy little board books&#8212;with their bright, simple pictures, easy text, and hand&#8211;friendly formats&#8212;are just the thing to delight the very young. Ages 6 months&#8211;2 years.'}
{'title': 'Girls Ballet Tutu Neon Blue', 'content': 'Dance tutu for girls ages 2-8 years. Perfect for dance practice, recitals and performances, costumes or just for fun!'}
{'title': 'The Prophet', 'content': 'In a distant, timeless place, a mysterious prophet walks the sands. At the moment of his departure, he wishes to offer the people gifts but possesses nothing. The people gather round, each asks a question of the heart, and the

In [9]:
# print the last 20 rows of the dataset

dataset_length = len(dataset)
for i in range(dataset_length - 20, dataset_length):
  print(dataset['train'][i])

{'title': 'Dreamworks Dragons 2 Backpack - Blue', 'content': "This DreamWorks How to Train Your Dragon 2 Boy's 16 inch Backpack - Blue features a zip up front pocket and 2 mesh side pockets in addition to the main storage compartment."}
{'title': 'Bubble Design Soft Silicom Waterproof Swimming Cap for Long Hair Women and Men', 'content': 'Good quality and soft silicon material,more comfortable when you swimming.With high elasticity,can extend to 2 times of the oringinal size,and fit the head more tightly.Round shap design make it low Resistance when you are in the water.Use the cool water clean the cap after every use,and after that pls dry the cap.Note:1.When the cap is waterish,pls do not roll up the cap.2.To keep the cap durable,Pls donnt keep this cap long time stay behind the sun.3.Pls do not make the cap touch any sharps.4.When will keep the cap for long time no use,pls spread some talcum powder.'}
{'title': 'Perfecto 100% Pure Badger Shaving Brush', 'content': 'The PerfectoTM Sh

In [10]:
# clean the dataset variable, removing html entities and lower casing both columns, title and content

def clean_text(text):
  text = BeautifulSoup(text, "html.parser").get_text()  # Remove HTML entities
  text = re.sub(r"[^a-zA-Z0-9,.?!\s]", "", text) # Remove special chars
  return text

def clean_dataset(dataset):
  cleaned_dataset = dataset.map(lambda example: {
      'title': clean_text(example['title']),
      'content': clean_text(example['content'])
  })
  return cleaned_dataset

dataset = clean_dataset(dataset)
print(dataset)

Map:   0%|          | 0/1390403 [00:00<?, ? examples/s]

<ipython-input-10-5775a35d3042>:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()  # Remove HTML entities
<ipython-input-10-5775a35d3042>:4: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()  # Remove HTML entities


DatasetDict({
    train: Dataset({
        features: ['title', 'content'],
        num_rows: 1390403
    })
})


In [11]:
# print the first 20 rows from the dataset variable
for i in range(20):
  print(dataset['train'][i])

{'title': 'Girls Ballet Tutu Neon Pink', 'content': 'High quality 3 layer ballet tutu. 12 inches in length'}
{'title': 'Mogs Kittens', 'content': 'Judith Kerrs bestselling adventures of that endearing and exasperating cat Mog have entertained children for more than 30 years. Now, even infants and toddlers can enjoy meeting this loveable feline. These sturdy little board bookswith their bright, simple pictures, easy text, and handfriendly formatsare just the thing to delight the very young. Ages 6 months2 years.'}
{'title': 'Girls Ballet Tutu Neon Blue', 'content': 'Dance tutu for girls ages 28 years. Perfect for dance practice, recitals and performances, costumes or just for fun!'}
{'title': 'The Prophet', 'content': 'In a distant, timeless place, a mysterious prophet walks the sands. At the moment of his departure, he wishes to offer the people gifts but possesses nothing. The people gather round, each asks a question of the heart, and the mans wisdom is his gift. It is Gibrans gift t

In [12]:
# Split the dataset into minor datasets with 20000 rows each, and save in the disk

dataset_length = len(dataset['train'])
rows_per_dataset = 20000
num_datasets = math.ceil(dataset_length / rows_per_dataset)

for i in range(num_datasets):
  start_index = i * rows_per_dataset
  end_index = min((i + 1) * rows_per_dataset, dataset_length)

  # Extract the subset directly as a Dataset object
  subset = dataset['train'].select(range(start_index, end_index))

  # Now you can save the subset to a new file
  subset.to_json(f'{BASE_PATH}/dataset_part_{i+1}.json')
  print(f"Dataset part {i+1} saved with {len(subset)} rows.")

Creating json from Arrow format:   0%|          | 0/20 [00:00<?, ?ba/s]

Dataset part 1 saved with 20000 rows.


Creating json from Arrow format:   0%|          | 0/20 [00:00<?, ?ba/s]

Dataset part 2 saved with 20000 rows.


Creating json from Arrow format:   0%|          | 0/20 [00:00<?, ?ba/s]

...

Dataset part 68 saved with 20000 rows.


Creating json from Arrow format:   0%|          | 0/20 [00:00<?, ?ba/s]

Dataset part 69 saved with 20000 rows.


Creating json from Arrow format:   0%|          | 0/11 [00:00<?, ?ba/s]

Dataset part 70 saved with 10403 rows.
